In [2]:
from sklearn.tree import DecisionTreeClassifier

X = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]
y = [1, 0, 0, 1, 1]

model = DecisionTreeClassifier(random_state=0)
model.fit(X, y)

print(model.predict([[5, 5]]))

[0]


In [ ]:
# 数据读取与预处理
import pandas as pd
df = pd.read_excel('员工离职预测模型.xlsx')
df.head()
# Python数学建模无法识别文本内容
df.replace({'工资': {'低': 0, '中': 1, '高': 2}})
df.head()
# 提取特征变量和目标变量
X = df.drop(columns='离职')
y = df['离职']
# 划分训练集和测试集（用来检验模型训练的结果），一般是八二分或者七三分
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2, random_state=123)
# 模型训练及搭建
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=3, random_state=123)
model.fit(X_train, y_train)
# 模型预测及评估
y_pred = model.predict(X_test)
a = pd.DataFrame() # 创建一个空的DataFrame
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)

from sklearn.metrics import accuracy_score
# score输出结果是0.9573，即3000个测试集数据中，有2872人预测结果和实际结果相符
score = accuracy_score(y_pred, y_test) 

# 分类决策树模型本质预测的并不是准确的0或1的分类，而是预测其属于某一分类的概率
# y_pred_proba是一个二维数组，其左侧一列数为分类为0的概率，右侧一列数为分类为1的概率
# 查看预测属于各个分类的概率
y_pred_proba = model.predict_proba(X_test)
b = pd.DataFrame(y_pred_proba, columns=['不离职概率', '离职概率'])
# 对于分类模型而言，不仅关心其预测的准确度，更关心两个指标：
# 命中率（TPR）：所有实际离职的员工中被预测为离职的比率
# 假警报率（FPR）：所有实际不离职的员工中被预测为离职的比率
# 阈值（thres）
# 即通过两者绘制的ROC曲线来评判模型
from sklearn.metrics import roc_curve
fpr, tpr, thres = roc_curve(y_test,  y_pred_proba[:,1])
# 可以查看不同阈值下的命中率和假警报率
a = pd.DataFrame() # 创建一个空DataFrame
a['阈值'] = list(thres)
a['命中率'] = list(tpr)
a['假警报率'] = list(fpr)
# 模型的AUC值
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, y_pred_proba[:,1])

# 已知不同阈值下的假警报率和命中率，可通过matplotlib库来绘制ROC曲线
import matplotlib.pyplot as plt
plt.plot(fpr, tpr)
plt.show()

# 特征重要性评估
# 整理特征重要性，方便结果呈现
features = X.columns # 获取特征名称
importances = model.feature_importances_ # 获取特征重要性
# 通过二维表格形式显示
importances_df = pd.DataFrame()
importances_df['特征名称'] = features
importances_df['特征重要性'] = importances
importances_df.sort_values('特征重要性', ascending=False)

# 这三行
# importances_df = pd.DataFrame()
# importances_df['特征名称'] = features
# importances_df['特征重要性'] = importances
# 也可以写成：
importances_df = pd.DataFrame([feature, importances], index=['特征名称', '特征重要性'])

# 如果想将决策树模型可视化展示出来，可使用Python的graphviz插件
# 模型可视化呈现只要是为了演示和教学，在真正实战中应用较少

![](./decision-tree.png)

根节点根据满意度是否<=4.65作为分类依据，
它当前基尼系数是0.365，样本数为12000，
value的第一个值9120表示是否离职中的0的分类，第一个值2880表示是否离职中的1的分类，
class表示分类结果，这时9120不离职比2880离职大，所以分类结果是不离职。
不过根节点的分类结果没有意义，主要看最后叶子节点的分类。
叶子节点基尼系数是0时，证明纯度已经很高，虽然有基尼系数不是0的叶子节点，为了控制过拟合，限制树的最大深度是3